### Install & Import & Define

In [1]:
! pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain
! pip install deep-translator


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os
import sys
from pathlib import Path
from typing import List, Any
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.llms import Ollama
from langchain.docstore.document import Document
from deep_translator import GoogleTranslator

In [116]:
BOOK_PATH = "../../downloaded_books/txt/the_heart_of_a_dog.txt"

### Splitting book

In [117]:
path = Path(BOOK_PATH)

In [118]:
with open(path, "r") as f:
    data = f.read()

In [119]:
len(data)

41811

In [9]:
def split_data(data: str, max_symbols) -> List[str]:
    parts = []
    for i in range(0, len(data), max_symbols):
        parts.append(data[i:i+max_symbols])
    return parts

In [121]:
parts = split_data(data, 4000)
len(parts[0]), len(parts[-1])

(4000, 1811)

### Summarize function

In [45]:
def get_summaries(parts: List[str], chain) -> List[str]:
    res = chain.invoke([Document(page_content=part) for part in parts])
    print(res.keys())
    print(len(res["input_documents"]))
    print(res["input_documents"][1].page_content[:100], res["input_documents"][2].page_content[:100])
    return res["output_text"]

In [36]:
docs = chain.invoke([Document(page_content="kek")])["input_documents"]

NameError: name 'chain' is not defined

### Summarize logic

In [46]:
def recursive_summarization(data: str, chain: Any, max_symbols=4000) -> str:
    print(f"len(data) = {len(data)}, wanted <= {max_symbols}")
    if len(data) == 0:
        return data
    parts = split_data(data, max_symbols)
    if len(parts) == 1:
        return parts[0]
    parts_summaries = get_summaries(parts, chain)
    return recursive_summarization(''.join(parts_summaries), chain, max_symbols)

### Function for summarize book

In [47]:
def get_book_summary(book_path: str) -> str:
    with open(book_path, "r") as f:
        data = f.read()
    print(data[:100])
    llm = Ollama(model="llama2")
    chain = load_summarize_chain(llm, chain_type="stuff")
    summary = recursive_summarization(data, chain)
    print(summary)
    # return summary
    translated = GoogleTranslator(source='auto', target='ru').translate(summary)
    return translated

In [48]:
get_book_summary("../../downloaded_books/txt/and_quiet_flows_the_don.txt")

Annotation


Роман-эпопея Михаила Шолохова «Тихий Дон» – одно из наиболее значительных, масштабных и
len(data) = 703955, wanted <= 4000
dict_keys(['input_documents', 'output_text'])
176
го по хутору чудно́е. Ребятишки, пасшие за прогоном телят, рассказывали, будто видели они, как Проко а.





* * *


Жена Прокофия умерла вечером этого же дня. Недоношенного ребенка, сжалившись, взяла 
len(data) = 130, wanted <= 4000

The summary is not available for this book as it is an excerpt from a larger work and does not have a concise summary on its own.


'Краткое изложение для этой книги недоступно, поскольку оно представляет собой отрывок из более крупной работы и не содержит отдельного краткого изложения.'

In [24]:
for path in Path("../../downloaded_books/txt/").glob("*.txt"):
    print(f"Getting summary of {path.stem}")
    summary = get_book_summary(path)
    print(f"Summary of {path.stem} = '{summary}'")
    dir = Path("../../downloaded_books/stupid_summaries_separate_chain/")
    dir.mkdir(parents=True, exist_ok=True)
    with open(f"{dir}/{path.stem}.txt", "w") as f:
        f.write(summary)

Getting summary of doctor_zhivago
len(data) = 105968, wanted <= 4000
len(data) = 669, wanted <= 4000
Summary of doctor_zhivago = 'Краткое содержание предоставленного текста:

Лара — молодая женщина, работающая официанткой в ​​ресторане. Она недовольна своей жизнью и чувствует себя в ловушке из-за своей работы и отношений со своим парнем Комаровским. Она мечтает уйти с работы и начать новую жизнь, но не может принять меры из-за финансовой зависимости от Комаровского. Она чувствует себя задыхающейся от его собственнического и контролирующего поведения и жаждет свободы и независимости. Несмотря на свое несчастье, Лара остается пассивной и принимает свою ситуацию, чувствуя себя бессильной ее изменить. Текст представляет собой обзор жизни и эмоций Лары, подготавливая почву для остальной части истории.'
Getting summary of evenings_on_a_farm_near_dikanka
len(data) = 132973, wanted <= 4000
len(data) = 1826, wanted <= 4000
Summary of evenings_on_a_farm_near_dikanka = 'Это list 48 Russian words 